In [ ]:
from kgx import Transformer, NeoTransformer

In [ ]:
# Monarch-Lite

# Credentials can be found from 'Registry of Biolink-compatible Neo4 instances' spreadsheet
monarch_host = ''
ports = {'bolt': 7687}
monarch_username = ''
monarch_password = ''

# Initialize NeoTransformer
monarch_lite_transformer = NeoTransformer(host=monarch_host, ports=ports, username=monarch_username, password=monarch_password)

# Define filters
monarch_lite_transformer.set_filter("subject_category", "gene")
monarch_lite_transformer.set_filter("object_category", "disease")
monarch_lite_transformer.set_filter("edge_label", "contributes_to")

# Load nodes and edges from remote Monarch-Lite Neo4j instance
start = 0
end = 20000
monarch_lite_transformer.load(start=start, end=end)
monarch_lite_transformer.report()

In [ ]:
# SemMedDB

# Credentials can be found from 'Registry of Biolink-compatible Neo4 instances' spreadsheet
semmeddb_host = ''
ports = {'bolt': 7687}
semmeddb_username = ''
semmeddb_password = ''

# Initialize NeoTransformer
semmeddb_transformer = NeoTransformer(host=semmeddb_host, ports=ports, username=semmeddb_username, password=semmeddb_password)

# Define filters
semmeddb_transformer.set_filter("subject_category", "gene")
semmeddb_transformer.set_filter("object_category", "chemical_substance")
semmeddb_transformer.set_filter("edge_label", "coexists_with")

# Load nodes and edges from remote SemMedDB Neo4j instance
start = 0
end = 20000
semmeddb_transformer.load(start=start, end=end)
semmeddb_transformer.report()

In [ ]:
# First, we perform a remap operation to bring SemMedDB gene nodes to HGNC namespace
semmeddb_transformer.remap_node_identifier("gene", "xrefs", "HGNC:")

In [ ]:
# Then we merge Monarch-Lite graph with SemMedDB graph
graphs = [monarch_lite_transformer.graph, semmeddb_transformer.graph]
merged_transformer = Transformer()
merged_transformer.merge(graphs)

In [ ]:
# Now, we get a gene common in both the graphs
rad51_gene_node = [n for n in merged_transformer.graph.nodes(data=True) if n[1]['id'] == 'HGNC:9817'][0]
rad51_gene_node

In [ ]:
# we get all out-going edges from RAD51 gene node
out_edges = merged_transformer.graph.out_edges(rad51_gene_node[0], data=True)

# we can see that there are 5 edges going to disease nodes (from Monarch-Lite) 
# and 7 edges going to chemical_substance nodes
out_edges